In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import matplotlib.pyplot as plt



## Zeitserie erstellen

In [3]:
def generate_time_eries(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # Welle 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # Welle 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # Rauschen
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_eries(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [1]:
model = keras.models.Sequential([
    keras.layers.Flatten(input=(50, 1)),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


NameError: name 'keras' is not defined

### Das einfachste RNN das man bauen könnte

In [4]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


/Users/riccardo/Desktop/Github/Bachelor/Bachelor/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


---------------------------------------------------------------------------

# Mehrere Zeitschritte vorhersagen

In [2]:
series = generate_time_eries(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

y_pred = X[:, n_steps:]




NameError: name 'generate_time_eries' is not defined

# Recurrent Neural Network

## Vorhersage des DAX Kurses

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("data/raw/^GDAXI.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0.0
1,1987-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2,1988-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0.0
4,1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0.0


In [8]:
print(df.describe())
df.isna().sum()

               Open          High           Low         Close     Adj Close  \
count   9213.000000   9213.000000   9213.000000   9213.000000   9213.000000   
mean    6710.231490   6755.542185   6660.681012   6709.836470   6709.836470   
std     4387.626710   4411.600909   4362.377131   4387.967762   4387.967762   
min      936.000000    936.000000    936.000000    936.000000    936.000000   
25%     2821.320068   2850.939941   2786.610107   2820.810059   2820.810059   
50%     5812.810059   5869.220215   5761.299805   5814.740234   5814.740234   
75%     9905.669922   9978.940430   9836.459961   9920.919922   9920.919922   
max    18874.849609  18892.919922  18719.810547  18869.359375  18869.359375   

             Volume  
count  9.213000e+03  
mean   6.852572e+07  
std    6.247492e+07  
min    0.000000e+00  
25%    0.000000e+00  
50%    7.117340e+07  
75%    1.044807e+08  
max    5.101956e+08  


Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
df.isna().sum()
df = df.dropna()